In [1]:
!pip install gradio openai-whisper gtts deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 106.8 MB/s e

In [2]:
import gradio as gr
import whisper
from gtts import gTTS
from deep_translator import GoogleTranslator
import uuid
from pathlib import Path

In [8]:

# Function to transcribe audio using Whisper
def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result['text']


# Function to translate text using GoogleTranslator
def translate_text(text: str) -> list:
    languages = ["ru", "tr", "zh-CN", "de", "es", "ja"]
    list_translations = []

    for lang in languages:
        translation = GoogleTranslator(source="auto", target=lang).translate(text)
        list_translations.append(translation)

    return list_translations


# Function to generate speech from text using gTTS
def text_to_speech(text, lang):
    tts = gTTS(text=text, lang=lang)
    filename = f"{uuid.uuid4()}.mp3"
    tts.save(filename)
    return filename


# Main function to handle voice-to-voice translations
def voice_to_voice(audio_file):
    # Transcribe speech using Whisper
    transcript = transcribe_audio(audio_file)

    # Translate text
    translations = translate_text(transcript)

    # Generate speech from translated text using gTTS
    audio_files = []
    for translation, lang in zip(translations, ["ru", "tr", "zh-CN", "de", "es", "ja"]):
        audio_file = text_to_speech(translation, lang)
        audio_files.append(audio_file)

    # Return audio files and translations
    return (audio_files[0], audio_files[1], audio_files[2], audio_files[3], audio_files[4], audio_files[5],
            translations[0], translations[1], translations[2], translations[3], translations[4], translations[5])


# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Запишите свой голос и получайте переводы с воспроизведением.")

    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Record your voice")

    with gr.Row():
        submit = gr.Button("Submit")
        btn = gr.ClearButton(audio_input, "Clear")

    with gr.Row():
        with gr.Group():
            ru_audio = gr.Audio(label="Russian")
            ru_text = gr.Textbox(label="Russian Text")

        with gr.Group():
            tr_audio = gr.Audio(label="Turkish")
            tr_text = gr.Textbox(label="Turkish Text")

        with gr.Group():
            zh_CN_audio = gr.Audio(label="Chinese")
            zh_CN_text = gr.Textbox(label="Chinese Text")

        with gr.Group():
            de_audio = gr.Audio(label="German")
            de_text = gr.Textbox(label="German Text")

        with gr.Group():
            es_audio = gr.Audio(label="Spanish")
            es_text = gr.Textbox(label="Spanish Text")

        with gr.Group():
            ja_audio = gr.Audio(label="Japanese")
            ja_text = gr.Textbox(label="Japanese Text")

    output_components = [ru_audio, tr_audio, zh_CN_audio, de_audio, es_audio, ja_audio,
                         ru_text, tr_text, zh_CN_text, de_text, es_text, ja_text]

    submit.click(fn=voice_to_voice, inputs=audio_input, outputs=output_components)

# Run the Gradio app in Colab
if __name__ == "__main__":
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57c6441d0148a529eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
